In [1]:
import torch
from model import Model
import matplotlib.pyplot as plt
import torchvision.transforms as T 
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Model(device=device)

path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'

noisy_imgs_1, noisy_imgs_2 = torch.load(path_train)
noisy_imgs_1 = noisy_imgs_1
noisy_imgs_2 = noisy_imgs_2

noisy_imgs , clean_imgs = torch.load(path_val)
# noisy_imgs = noisy_imgs.float()/256
# clean_imgs = clean_imgs.float()/256


/idiap/temp/sshetty/miniconda/envs/pyml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-8abppaug because the default path (/idiap/home/sshetty/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:

model.train(noisy_imgs_1, noisy_imgs_2, 100)
out = model.predict(noisy_imgs)

  6%|█████████▏                                                                                                                                               | 6/100 [00:22<05:57,  3.81s/it]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.70 GiB total capacity; 15.60 GiB already allocated; 16.69 MiB free; 17.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
transform = T.ToPILImage()
plt.figure(figsize = [10, 5])
plt.subplot(1, 3,1)
plt.imshow(transform(noisy_imgs[20]));
plt.subplot(1, 3,2)
plt.imshow(transform(out[20]/256));
plt.subplot(1, 3,3)
plt.imshow(transform(clean_imgs[20]));

In [ ]:
def psnr(denoised, ground_truth):
    mse = torch.mean((denoised-ground_truth)**2)
    return -10 * torch.log10(mse + 10**(-8))

psnr(out.cpu().float()/256, clean_imgs.float()/256)


In [ ]:

import matplotlib.pyplot as plt
plt.plot(mod.loss_train.detach().cpu())
plt.plot(mod.loss_valid.detach().cpu())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.loss_train.detach().cpu())